In [79]:
import pickle
from Levenshtein import distance, ratio
import re
import json
from autocorrect import Speller
from IPython.display import HTML, display
spell = Speller(lang='en')

with open('/projects/0/prjs0809/Daniel/DSP/data/posts.pickle', '+rb') as f:
    data = pickle.load(f)

with open('drug_dictionary.json') as f:
    drug_dict = json.load(f)

In [92]:
def visualize_ner(text, tags):
    # Define colors for entity types
    color_dict = {'DRUG': 'orange'}

    # Start with the raw text
    html = f"<p style='font-size:16px;'>"

    # Go through each word and its tag
    for word, tag in zip(text.split(), tags):
        if tag != 'O':  # 'O' usually represents 'no entity'
            color = color_dict.get(tag, 'yellow')
            html += f"<mark style='background-color: {color};'>{word}</mark> "
        else:
            html += word + " "
    
    html += "</p>"
    return display(HTML(html))

In [82]:
# add actual drug names to the list (keys of drug dict)

drug_keys = list(drug_dict.keys())
drug_values = list(set(drug_dict.values()))
drug_list = drug_keys + drug_values

In [83]:
drug_list_clean = [re.sub(r'[^a-zA-Z0-9 ]', '', drug) for drug in drug_list]

In [84]:
len(data)

30047

In [88]:
import tqdm

labeled_posts = []
labeled_words = []

spell = Speller(lang='en')
sentence_num = 1

for i, post in tqdm.tqdm(enumerate(data)):
    comment = post['body']
    if len(comment.split()) > 20 or len(comment.split()) < 3:
        continue

    comment = comment.lower()
    comment = re.sub(r'\n', '', comment) # remove new lines
    comment = re.sub(r'http\S+|www\S+|https\S+', '', comment, flags=re.MULTILINE) # remove links
    # comment = re.sub(r'[^\w\s]', '', comment) # remove punctuation
    comment = re.sub(r'\d+', '', comment) # remove numbers

    occurrences = []
    for expression in drug_list:
        if re.search(r'\b' + re.escape(expression) + r'\b', comment):
            corrected_expression = spell(expression)
            occurrences.append(corrected_expression)

    
    labels = ['B-drug' if word in occurrences else 'O' for word in comment.split()]

    if len(occurrences) > 0:
        for j, word in enumerate(comment.split()):
            if j == 0:
                labeled_words.append({'Sentence #': f'Sentence: {sentence_num}', 'Word': word, 'Tag': labels[j]})
                sentence_num += 1
            else:
                labeled_words.append({'Word': word, 'Tag': labels[j]})

        labeled_posts.append([comment, labels])


   




30047it [08:11, 61.15it/s] 


In [89]:
import pandas as pd
dataframe = pd.DataFrame(labeled_words)

In [90]:
dataframe

,Sentence #,Word,Tag
0,Sentence: 1,kobb,O
1,NaN,glass,B-drug
2,NaN,on,O
3,NaN,insta,O
4,NaN,makes,O
...,...,...,...
37269,NaN,help,O
37270,NaN,give,O
37271,NaN,you,O
37272,NaN,an,O


In [91]:
dataframe.to_csv('labeled_posts.csv', index=False)

In [93]:
for i in range(len(labeled_posts)):
    visualize_ner(*labeled_posts[i])
